## normalized burn ratio


In [1]:
workflow = dict([('id', 'nbr'),
                ('label', 'Normalized burn ratio'),
                ('doc', 'Normalized burn ratio processor')])


In [26]:
input_reference = dict([('id', 'input_reference'), 
                        ('label', 'EO product for normalized burn ratio'),
                        ('doc', 'EO product for normalized burn ratio'),
                        ('value', 'https://catalog.terradue.com/sentinel2/search?uid=S2B_MSIL2A_20190612T095039_N0212_R079_T33TVF_20190612T124245'), 
                        ('stac:collection', 'input_reference'),
                        ('stac:href', '/workspace/data/s2/catalog.json'),
                        ('type', 'Directory'),
                        ('scatter', 'True')])

In [3]:
import os
import sys
import gdal
import numpy as np
import logging
from pystac import Catalog, Collection, EOItem, MediaType, EOAsset, CatalogType

gdal.UseExceptions()

if not 'PREFIX' in os.environ.keys():
    
    os.environ['PREFIX'] = '/opt/anaconda/envs/env_nbr/'

os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
cat = Catalog.from_file(input_reference['stac:href'])

In [6]:
collection = next(cat.get_children())

In [7]:
item = next(collection.get_items())

In [8]:
for index, band in enumerate(item.bands):
   
    if band.common_name in ['swir', 'swir22']:
 
       asset_swir = item.assets[band.name].get_absolute_href()
        
    if band.common_name in ['nir']:
 
        asset_nir = item.assets[band.name].get_absolute_href()

In [9]:
vrt = '{0}.vrt'.format(item.id)

In [10]:
ds = gdal.BuildVRT(vrt,
                   [asset_nir, asset_swir],
                   srcNodata=0,
                   xRes=10, 
                   yRes=10,
                   separate=True)

ds.FlushCache()

ds = None

del(ds)


In [11]:
tif = '{0}.tif'.format(item.id)

In [12]:
gdal.Translate(tif,
               vrt,
               outputType=gdal.GDT_Int16)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7ff0a06a1870> >

In [13]:
os.remove(vrt)

In [14]:
ds = gdal.Open(tif)
width = ds.RasterXSize
height = ds.RasterYSize

input_geotransform = ds.GetGeoTransform()
input_georef = ds.GetProjectionRef()

In [15]:
nir = ds.GetRasterBand(1).ReadAsArray()
swir = ds.GetRasterBand(2).ReadAsArray()

In [16]:
nbr = np.zeros((height, width), dtype=np.float)

nbr = (nir - swir) / (nir + swir)

/opt/anaconda/envs/env_nbr/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
nir = swir = None

In [18]:
output_name = 'NRB_{}.tif'.format(item.id)

In [19]:
driver = gdal.GetDriverByName('GTiff')
    
output = driver.Create(output_name, 
                       width, 
                       height, 
                       1, 
                       gdal.GDT_Float32)

output.SetGeoTransform(input_geotransform)
output.SetProjection(input_georef)
output.GetRasterBand(1).WriteArray(nbr),

output.FlushCache()

output = None

del(output)

os.remove(tif)

## Declaring the results

In [20]:
catalog = Catalog(id='catalog', description='Results')

catalog.clear_items()
catalog.clear_children()

<Catalog id=catalog>

In [21]:
bands = [{"name": "NBR",
          "common_name": "nbr"
         }]

    
result_item = EOItem(id=output_name,
                   geometry=item.geometry,
                   bbox=item.bbox,
                   datetime=item.datetime,
                   properties={},
                   bands=bands,
                    gsd=10, 
                    platform=item.platform, 
                    instrument=item.instrument)

result_item.add_asset(key='data',
                      asset=EOAsset(href='./{}'.format(output_name), 
                      media_type=MediaType.GEOTIFF, 
                      title='Normalized burn ratio',
                     bands=bands[0]))


    
catalog.add_items([result_item])

In [22]:
catalog.normalize_and_save(root_href='stac-results',
                           catalog_type=CatalogType.SELF_CONTAINED)

In [23]:
catalog.describe()

* <Catalog id=catalog>
  * <EOItem id=NRB_S2B_MSIL2A_20200130T004659_N0213_R102_T53HPA_20200130T022348.tif>
